# Задание

Обучить модель суммаризации на основе данных и предобученной модели из открытого доступа на русском языке. Нельзя брать датасет, если выбранная модель на нём обучалась.

- Выбрать модель: https://huggingface.co/models
- Выбрать данные: https://huggingface.co/datasets

В качестве результата должны быть представлены примеры суммаризации из тестовой части выбранного набора данных для исходной и дообученой модели.

In [1]:
import torch
import numpy as np
from torch import optim
from torch.utils.data import DataLoader
from torchmetrics.functional.text import bleu_score
from transformers import T5Tokenizer, T5ForConditionalGeneration
import lightning.pytorch as pl
from datasets import load_dataset
from lightning.pytorch.loggers import TensorBoardLogger

In [2]:
import os
os.environ['http_proxy'] = 'http://proxy.stc:3128'
os.environ['https_proxy'] = 'http://proxy.stc:3128'
os.environ['ftp_proxy'] = 'http://proxy.stc:3128'

In [9]:
class Seq2Seq(pl.LightningModule):
    def __init__(self, model_name_or_path):
        super().__init__()
        self.save_hyperparameters()
        self.model = T5ForConditionalGeneration.from_pretrained(model_name_or_path)
        self.tokenizer = T5Tokenizer.from_pretrained(model_name_or_path)
        
    def forward(self, x):
        
        return self.model(x)
    
    def training_step(self, batch, batch_idx):
        
        loss = self.model(input_ids=batch["input_ids"].to(device="cuda"), labels=batch["labels"].to(device="cuda")).loss
        self.log("train_loss", loss.item(), prog_bar=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        val_loss = self.model(input_ids=batch["input_ids"].to(device="cuda"), labels=batch["labels"].to(device="cuda")).loss
        self.log("val_loss", val_loss.item(), prog_bar=True)
        return val_loss
    
    def test_step(self, batch, batch_idx):
        outputs = self.model.generate(batch["input_ids"].to(device="cuda"), max_new_tokens=80)
        
        preds = []
        for i in range(len(outputs)):
            pred = self.tokenizer.decode(outputs[i], skip_special_tokens=True)
            preds.append(pred)
        
        bleu = bleu_score(preds, batch["summaries"])
        self.log("bleu", bleu.item(), prog_bar=True)
        return bleu
    
    def predict_step(self, batch, batch_idx):
        #print(batch)
        outputs = self.model.generate(batch["input_ids"].to(device="cuda"), max_new_tokens=80)

        preds = []
        for i in range(len(outputs)):
            pred = self.tokenizer.decode(outputs[i], skip_special_tokens=True)
            preds.append(pred)
        preds = np.array(preds) 
        
        return preds

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=2e-5)
        return optimizer

In [10]:
class Seq2SeqDataModule(pl.LightningDataModule):
    def __init__(self, model_name_or_path, ds_name):
        super().__init__()
        self.tokenizer = T5Tokenizer.from_pretrained(model_name_or_path)
        self.ds_name = ds_name
    
        
    def prepare_data(self):
        self.ds = load_dataset(self.ds_name)
        for split in self.ds:
            #if split == "train":
                #self.ds[split] = self.ds[split].select(list(range(100)))
            self.ds[split] = self.ds[split].select(list(range(1000)))  
            self.ds[split] = self.ds[split].map(self.tokenize)
        
    def train_dataloader(self):
        train_split = self.ds["train"]
        return DataLoader(train_split, batch_size=2, collate_fn=self.collate, num_workers=os.cpu_count())
    
    def val_dataloader(self):
        val_split = self.ds["validation"]
        return DataLoader(val_split, batch_size=2, collate_fn=self.collate, num_workers=os.cpu_count())
    
    def test_dataloader(self):
        test_split = self.ds["test"]
        return DataLoader(test_split, batch_size=4, collate_fn=self.collate_test, num_workers=os.cpu_count())
    
    def predict_dataloader(self):
        test_split = self.ds["test"]
        return DataLoader(test_split, batch_size=4, collate_fn=self.collate_test, num_workers=os.cpu_count())
    
    def tokenize(self, row):
        row["input_ids"] = self.tokenizer(f'summarize: {row["text"]}', max_length=512, padding='max_length', truncation=True, return_tensors="pt").input_ids.squeeze()
        row["labels"] = self.tokenizer(row["summary"], max_length=512, padding='max_length', truncation=True, return_tensors="pt").input_ids.squeeze()
        return row
    
    def collate(self, samples):
        input_ids = torch.stack([torch.tensor(s["input_ids"]) for s in samples])
        labels = torch.stack([torch.tensor(s["labels"]) for s in samples])
        
        return {"input_ids": input_ids, "labels": labels}
    
    def collate_test(self, samples):
        summaries = np.stack([np.array(s["summary"]) for s in samples])
        input_ids = torch.stack([self.tokenizer(f'summarize: {s["text"]}', max_length=512, padding='max_length', truncation=True, return_tensors="pt").input_ids.squeeze() for s in samples])
        
        return {"input_ids": input_ids, "summaries": summaries}
        

In [11]:
seq2seq_model = Seq2Seq("cointegrated/rut5-base-multitask")
#seq2seq_model = Seq2Seq("csebuetnlp/mT5_multilingual_XLSum")
#seq2seq_model = Seq2Seq("cointegrated/rut5-base-absum")

In [12]:
seq2seq_dm = Seq2SeqDataModule("cointegrated/rut5-base-multitask", "IlyaGusev/gazeta")
#seq2seq_dm = Seq2SeqDataModule("csebuetnlp/mT5_multilingual_XLSum", "IlyaGusev/gazeta")
#seq2seq_dm = Seq2SeqDataModule("cointegrated/rut5-base-absum", "IlyaGusev/gazeta")

In [13]:
trainer = pl.Trainer(max_epochs=2)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [14]:
trainer.predict(model=seq2seq_model, datamodule=seq2seq_dm)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |                                             | 0/? [00:00<?, ?it/s]

[array(['В Берлине прошли массовые акции протеста против антикоронавирусных мер',
        'США и Израиль в ОАЭ намерены обсудить вопросы двустороннего сотрудничества',
        'В Белоруссии КС объявили о создании оппозиционной партии «Вместе»',
        'Россия считает действия ВС США во время учений в Эстонии провокационными и крайне опасными для региональной стабильности'],
       dtype='<U120'),
 array(['В России вступают в силу поправки в закон «О банкротстве»',
        'Когда Бухарский эмират вновь обрел независимость',
        'Поклонская заявила, что Украина «вопиющим образом» нарушает права крымчан',
        'В Екатеринбурге скончался советский и российский писатель Владислав Крапивин'],
       dtype='<U76'),
 array(['В Минобороны РФ сообщили, что российский истребитель Су-27 не нарушал границ Дании',
        'Трамп назвал «марионеткой» своего соперника на предстоящих выборах американского лидера Джо Байдена',
        'Врачи из Лилльского университета во Франции рассказали, что 

In [15]:
#trainer.predict(model=seq2seq_model, datamodule=seq2seq_dm)

In [16]:
trainer.test(model=seq2seq_model, datamodule=seq2seq_dm)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |                                                | 0/? [00:00<?, ?it/s]

/home/ext-zorkina-a@ad.speechpro.com/.local/lib/python3.8/site-packages/lightning/pytorch/utilities/data.py:77: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 4. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│           bleu            │   0.0025510459672659636   │
└───────────────────────────┴───────────────────────────┘

[{'bleu': 0.0025510459672659636}]

In [17]:
trainer.fit(model=seq2seq_model, datamodule=seq2seq_dm)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 244 M 
-----------------------------------------------------
244 M     Trainable params
0         Non-trainable params
244 M     Total params
977.237   Total estimated model params size (MB)


Sanity Checking: |                                        | 0/? [00:00<?, ?it/s]

Training: |                                               | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=2` reached.


In [18]:
trainer.predict(model=seq2seq_model, datamodule=seq2seq_dm)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |                                             | 0/? [00:00<?, ?it/s]

[array(['В Берлине прошли массовые акции протеста против антикоронавирусных мер. В Берлине протестующие скандировали «Путин!» и скандировали «Путин!»',
        'США и Израиль намерены заключить историческое соглашение о нормализации отношений. В ходе визита в Абу-Даби стороны заключили историческое соглашение о нормализации отношений.',
        'Белорусская оппозиция Белоруссии намерена создать оппозиционную партию «Вместе». Это инициатива и инициатива Виктора Бабарико.',
        'Россия считает, что действия ВС США в Эстонии провокационными и крайне опасными для региональной стабильности. Российский посол считает, что действия ВС США в Эстонии провокационными и крайне опасными для региональной стабильности.'],
       dtype='<U231'),
 array(['В России вступают в силу поправки в закон «О банкротстве», а для физлиц и индивидуальных предпринимателей будет проведена процедура банкротства без участия суда и финансового управляющего.',
        'Когда Бухарский эмират вновь обрел независимост

In [19]:
trainer.test(model=seq2seq_model, datamodule=seq2seq_dm)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |                                                | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│           bleu            │   0.029492786154150963    │
└───────────────────────────┴───────────────────────────┘

[{'bleu': 0.029492786154150963}]